# 1 Imports and inputs

### 1.1 Import packages

In [ ]:
import io, os, sys, types
import datetime
import glob
import gdxpds as gp
import pandas as pd
import numpy as np

### 1.2 User inputs

In [ ]:
filetype_input = 'gdx' #Choose input file type: 'gdx' or 'csv'
gams_dir = 'C:/GAMS/win64/28.2'
markets = ['DayAhead'] #Choose from: 'DayAhead', 'Balancing', 'FullYear', 'Investment'
output_name = 'January_test'
output_type = ['csv', 'Excel'] #Choose desired output type, from: 'Excel' or 'csv' (or both)

### 1.3 Read which are relevant variables + files

In [ ]:
# select which variables should be displayed
include_file = pd.read_csv('.\input\include.csv', encoding='utf8')

# drop all variables that shall NOT be included and set the index to the
# variable names
include_file = include_file[include_file.include == 'YES']
del include_file['include']
include_file = include_file.set_index('variable')

# list of all variable names
var_list = list(include_file.index)

### 1.4 Create column names dictionary

In [ ]:
df_colnames = pd.read_csv('.\input\Dict_column_names.csv')
dict_colnames = dict(zip(df_colnames['Old'], df_colnames['New']))

# 2 Processing

### 2.1 Function 1: reading gdx-files

In [ ]:
def df_creation(gdx_file, varname):
    df = pd.DataFrame()
    if '_' in gdx_file:
            # if yes: extract scenario name from gdx filename
        scenario = gdx_file.split('_', 3)[-3]
        year = gdx_file.split('_', 3)[-2]
        subset = gdx_file.split('_', 3)[-1][:-4]
        market = gdx_file.split('\\', 1)[0].split('/',3)[-1]
    else:
           # if no: use nan instead
        scenario = 'nan'

    # create empty temporary dataframe and load the gdx data into it
    temp = pd.DataFrame()
    temp = gp.to_dataframe(gdx_file, varname, gams_dir=gams_dir,
                           old_interface=False)

    # add a scenario column with the scenario name of the current iteration
    temp['Scenario'] = scenario
    temp['Market']  = market
    temp['run'] = scenario + '_' + year + '_' + subset
    
    # rearrange the columns' order
    cols = list(temp.columns)
    cols = [cols[-1]] + cols[:-1]
    temp = temp[cols]

    # concatenate the temporary dataframe to the preceeding data
    df = pd.concat([df, temp], sort=False)
    return df

### 2.2 Use function 1 to read data

In [ ]:
runs = list()
gdx_file_list = list()

# directory to the input gdx file(s)
for market in markets:
    gdx_file_list = gdx_file_list + glob.glob('./input/results/'+ market + '/*.gdx')

all_df = {gdx_file : {varname: df for varname, df in zip(var_list,var_list)} for gdx_file in gdx_file_list}


for gdx_file in gdx_file_list:
    for varname, df in zip(var_list, var_list):
        all_df[gdx_file][varname] = df_creation(gdx_file, varname)
        if all_df[gdx_file][varname]['run'][0] not in runs:
            runs.append(all_df[gdx_file][varname]['run'][0])
            
run_dict = dict(zip(gdx_file_list, runs) )
all_df = dict((run_dict[key], value) for (key, value) in all_df.items())


### 2.3 Function 2: column aggregation

In [ ]:
#test function
def column_aggregator(input_df, settings):
    temp = input_df.copy()
    df_cond = settings.copy()
    output_df = pd.DataFrame()
    
    fields_in = list(df_cond['Fields_in'])
    fields_in = [x for x in fields_in if str(x) != 'nan']
    field_out = df_cond['Field_out'][0]

    nan_option = df_cond['nan_option'][0]
    nan_output = df_cond['nan_output'][0]

    dict_keys = list(df_cond['Keys']) 
    dict_keys = [x for x in dict_keys if str(x) != 'nan']
    
    dict_values = list(df_cond['Values'])
    dict_values = [x for x in dict_values if str(x) != 'nan']
    
    dictionary = dict(zip(dict_keys, dict_values))
    for i,row in temp.iterrows():
        temp.loc[i,'key'] = ''
        for field in fields_in:
            temp.loc[i,'key'] = temp.loc[i,'key'] + temp.loc[i,field]
        if field_out not in list(temp.columns):
            temp.loc[i,field_out] = np.NaN
            
    for j,row in temp.loc[temp['key'].isin(dict_keys), ].iterrows():     
        temp.loc[j, field_out] = dictionary[temp.loc[j,'key']]

    for k,row in temp.loc[temp[field_out].isna(), ].iterrows():
        if nan_option == 'field':
            temp.loc[k, field_out] = temp.loc[k, nan_output]
        if nan_option == 'string': 
            temp.loc[k, field_out] = nan_output
    output_df = output_df.append(temp)
    return output_df


### 2.4 Use function 2 to create output dataframes

In [ ]:
df_dict={i:pd.DataFrame() for i in var_list}
for var in var_list:
    if os.path.isfile('.\input\settings\settings_'+ var  +'.xlsx'):
        df_settings = pd.read_excel('.\input\settings\settings_'+ var  +'.xlsx', None)
    else:
        df_settings = pd.DataFrame()
        
    sheets = list(df_settings.keys())
    for run in runs:
        temp = all_df[run][var]
        for sheet in sheets:
            print('working on: ' + var + ', ' + run + ', ' + sheet)
            temp = column_aggregator(input_df = temp, settings = df_settings[sheet])
        df_dict[var] = df_dict[var].append(temp)

### 2.5 Change column names

In [ ]:
for var in var_list:
    cols = list(df_dict[var].columns)
    cols.insert(0,cols[cols.index('Market')])
    cols.insert(1,cols[cols.index('Scenario')])
    cols.remove('run')
    temp = pd.DataFrame()
    for i in cols:
        temp[i] = df_dict[var][i]
    df_dict[var] = temp
    df_dict[var] = df_dict[var].rename(columns = dict_colnames)

# 3 Output to CSV

In [ ]:
#Make output folder
if not os.path.isdir('output'):
    os.makedirs('output')

if 'csv' in output_type:
    for var in var_list:
        df_dict[var].to_csv('./output/' + output_name + '_' + var + '.csv', index = False)
             
if 'Excel' in output_type:
    with pd.ExcelWriter('./output/'+ output_name + '.xlsx') as writer:  
        for var in var_list:
            df_dict[var].to_excel(writer, sheet_name= var, index = False)
        